In [2]:
import psycopg2
from dotenv import load_dotenv
import os

# Load env vars
load_dotenv()

conn = psycopg2.connect(
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    dbname=os.getenv("PG_DB")
)

cur = conn.cursor()

schema_sql = """
CREATE TABLE IF NOT EXISTS countries (
    country_id SERIAL PRIMARY KEY,
    institute_country TEXT NOT NULL,
    num_institutes INT DEFAULT 0,
    total_programs INT DEFAULT 0,
    avg_ctr NUMERIC(5,2) DEFAULT 0.00,
    total_views BIGINT DEFAULT 0,
    total_impressions BIGINT DEFAULT 0,
    top_programs JSONB,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS institutions (
    institution_id SERIAL PRIMARY KEY,
    institution_name TEXT NOT NULL,
    country_id INT REFERENCES countries(country_id) ON DELETE CASCADE,
    institution_type TEXT,
    ctr NUMERIC(5,2) DEFAULT 0.00,
    total_views BIGINT DEFAULT 0,
    total_impressions BIGINT DEFAULT 0,
    top_programs JSONB,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE IF NOT EXISTS programs (
    program_id SERIAL PRIMARY KEY,
    program_name TEXT NOT NULL,
    institution_id INT REFERENCES institutions(institution_id) ON DELETE CASCADE,
    country_id INT REFERENCES countries(country_id) ON DELETE CASCADE,
    duration_months INT,
    tuition NUMERIC(12,2),
    ctr NUMERIC(5,2) DEFAULT 0.00,
    total_views BIGINT DEFAULT 0,
    total_impressions BIGINT DEFAULT 0,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

cur.execute(schema_sql)
conn.commit()


In [7]:
with conn.cursor() as cur:
    cur.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public';
    """)
    tables = cur.fetchall()

tables

[('countries',), ('institutions',), ('programs',)]

In [3]:
with conn.cursor() as cur:
    for table in ['countries', 'institutions', 'programs']:
        print(f"\nSchema for table {table}:")
        cur.execute(f"""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_name = '{table}';
        """)
        columns = cur.fetchall()
        for col in columns:
            print(f"{col[0]}: {col[1]}")


Schema for table countries:
updated_at: timestamp without time zone
country_id: integer
num_institutes: integer
total_programs: integer
avg_ctr: numeric
total_views: bigint
total_impressions: bigint
top_programs: jsonb
created_at: timestamp without time zone
institute_country: text

Schema for table institutions:
updated_at: timestamp without time zone
created_at: timestamp without time zone
institution_id: integer
country_id: integer
ctr: numeric
total_views: bigint
total_impressions: bigint
institution_name: text
institution_type: text

Schema for table programs:
updated_at: timestamp without time zone
program_id: integer
institution_id: integer
country_id: integer
duration_months: integer
tuition: numeric
ctr: numeric
total_views: bigint
total_impressions: bigint
created_at: timestamp without time zone
program_name: text


In [ ]:
cur = conn.cursor() 

alter_query = """
ALTER TABLE countries
ADD CONSTRAINT countries_institute_country_key UNIQUE (institute_country);
"""

cur.execute(alter_query)
conn.commit()
cur.close()

In [11]:
conn.rollback()

In [13]:
cur = conn.cursor()

cur.execute("""
    ALTER TABLE countries
    ALTER COLUMN top_programs TYPE jsonb
    USING to_jsonb(top_programs);
""")

conn.commit()
cur.close()

In [12]:
cur = conn.cursor()
cur.execute("""
    ALTER TABLE institutions DROP COLUMN IF EXISTS top_programs;
""")
conn.commit()
cur.close()